In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
base_path = '/content/drive/My Drive/DM Challenge'
save_path = base_path + '/poets_with_imgs.csv'

In [ ]:
import requests
source = 'http://famouspoetsandpoems.com'
page = requests.get(source+'/poets.html')
page

In [ ]:
from bs4 import BeautifulSoup
main_soup = BeautifulSoup(page.content, 'html.parser')
poets = main_soup.find('table', cellspacing='0').find('table').find_all('td', valign='top')
print (poets)

In [ ]:
!pip install deep-translator
from deep_translator import GoogleTranslator

In [ ]:
'''fields:
1. name
2. birth_year
3. death_year
4. bio
5. poem
6. quote
7. category (school)
8. similar poets
9. image
'''

'fields:\n1. name\n2. birth_year\n3. death_year\n4. bio\n5. poem\n6. quote\n7. category (school)\n8. similar poets\n9. image\n'

In [ ]:
from bs4 import NavigableString, Tag
import pandas as pd
import re
poets_df = pd.DataFrame(columns=['name', 'birth_year', 'death_year', 'categories', 'bio', 'poem', 'quote', 'similar_poets', 'image'])

for i in range(1000):
  try:
    td = poets[i]

    url = td.find('a')['href']
    text = td.text.split('(')

    name = text[0].strip()
    name_sin = GoogleTranslator(source='en', target='si').translate(name)
    birth_year = text[2].split('-')[0].strip()
    if not birth_year.isnumeric():
      birth_year = GoogleTranslator(source='en', target='si').translate(birth_year)
    death_year = text[2].split('-')[1].strip(')').strip()
    if not death_year.isnumeric():
      death_year = GoogleTranslator(source='en', target='si').translate(death_year)
    
    bio_page = requests.get('https://poets.org/poet/'+'-'.join(name.split()))
    bio_soup = BeautifulSoup(bio_page.content, 'html.parser')
    bio = bio_soup.find('div', class_='poet__body-content').text
    bio_sin = GoogleTranslator(source='en', target='si').translate(bio)

    try:
      img_page = requests.get(source+url+'/photo')
      img_soup = BeautifulSoup(img_page.content, 'html.parser')
      img = source+img_soup.find('img', alt=name)['src']
    except:
      img = 'http://famouspoetsandpoems.com/images/_no_photo.gif'

    categories = [c.text.strip() for c in bio_soup.find_all('div', class_='school')]
    categories_sin = ', '.join([GoogleTranslator(source='en', target='si').translate(c) for c in categories])
    
    similar_poets = [p.text.strip() for p in bio_soup.find_all('div', class_='poet__sidebar-related-poets-poet')]
    similar_poets_sin = ', '.join([GoogleTranslator(source='en', target='si').translate(p) for p in similar_poets])

    poems_page = requests.get(source+url+'/poems')
    poems_soup = BeautifulSoup(poems_page.content, 'html.parser')
    poem_url = poems_soup.find_all('table', width='436')[1].find_all('a')[0]['href']
    poem_page = requests.get(source+poem_url)
    poem_soup = BeautifulSoup(poem_page.content, 'html.parser')
    poem = (str(poem_soup.find('div', style='padding-left:14px;padding-top:20px;font-family:Arial;font-size:13px;')).strip('<div style="padding-left:14px;padding-top:20px;font-family:Arial;font-size:13px;">').strip('</div>').strip().split('<br/>'))
    poem_sin = '\n'.join([GoogleTranslator(source='en', target='si').translate(i) for i in poem if i!=''])
    
    quotes_page = requests.get(source+url+'/quotes')
    quotes_soup = BeautifulSoup(quotes_page.content, 'html.parser')
    quotes = quotes_soup.find_all('div', style='padding-right:15px;padding-left:16px;padding-bottom:20px;')
    if len(quotes)>0:
      quote = quotes[0].text
      quote_sin = GoogleTranslator(source='en', target='si').translate(quote)
    else:
      quote = None
      quote_sin = None
    
    if re.search('[a-zA-Z]', name_sin):
      print (name_sin)
      continue
    
    poets_df.loc[i] = [name_sin, birth_year, death_year, categories_sin, bio_sin, poem_sin, quote_sin, similar_poets_sin, img]
    print (i, name_sin)

  except:
    continue
  

In [ ]:
poets_df

In [ ]:
poets_df.to_csv(save_path)